In [1]:
!pip install -Uq transformers
!pip install -Uq evaluate
!pip install -Uq SentencePiece

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 61.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.8/294.8 kB 28.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 80.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 61.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.4/81.4 kB 665.5 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.6/519.6 kB 10.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 9.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 18.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 12.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 11.9 MB/s eta 0:00:00


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Load Configuration & Library

In [3]:
apipath = r'H:\\My Drive\\config\\hbqa.txt'
apipath = r'/content/drive/MyDrive/config/hbqa-colab.txt'
import configparser

config = configparser.ConfigParser()
config.read(apipath)
secret_key = config['global']['OPENAI_API_KEY']
datapath = config['global']['DATA_FOLDER']
corpuspath = config['global']['CORPUS_FOLDER']
PINECONE_API_KEY = config['global']['PINECONE_API_KEY']
PINECONE_ENV = config['global']['PINECONE_ENV']

# PINECONE is Vector Database. To store the vector so that we can quickly search the vector space.
# https://app.pinecone.io
# get PINECONE_API_KEY key from app.pinecone.io
# find your PINECONE_ENVIRONMENT next to the api key in pinecone console

In [4]:
import torch
import json
from tqdm import tqdm
import torch.nn as nn
from torch.optim import Adam
import nltk
import spacy
import string
import evaluate  # Bleu
from torch.utils.data import Dataset, DataLoader, RandomSampler
import pandas as pd
import numpy as np
import transformers
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from transformers import T5Tokenizer, T5Model, T5ForConditionalGeneration, T5TokenizerFast
import tensorflow as tf

import warnings
warnings.filterwarnings("ignore")

In [ ]:
# Detect and initialize TPU
tpu_available = tf.config.experimental.list_logical_devices("TPU")
if tpu_available:
    print("TPU available")
else:
    print("No TPU available")

In [5]:
import torch

if torch.cuda.is_available():
    DEVICE = torch.device("cuda")
else:
    DEVICE = torch.device("cpu")

In [6]:
DEVICE

device(type='cuda')

In [ ]:
# sample code
# from transformers import AutoTokenizer, AutoModelForQuestionAnswering

# TOKENIZER = AutoTokenizer.from_pretrained("hf-internal-testing/tiny-random-ReformerForQuestionAnswering")
# MODEL = AutoModelForQuestionAnswering.from_pretrained("hf-internal-testing/tiny-random-ReformerForQuestionAnswering")
# MODEL.to(DEVICE)

# Loading Dataset

In [7]:
import pandas as pd

In [8]:
df = pd.read_csv(datapath+"06-HBQA_Manual_with_Chunk.csv")

In [10]:
df.head(4)

,Ques_Id,Chunk_Id,Section_Id,Question,Ref_Answer,Chunk,Reference,WordsInQues,WordsInAns
0,10000,10193,Book01_046,Why did the Muni wander over the earth and wee...,The Muni wandered over the earth and wept loud...,"Sauti continued, 'The Muni, having said so unt...","""'O ye, being directed by my ancestors, I am r...",14,25
1,10001,10193,Book01_046,Who did the Muni ask for a bride and why?,The Muni asked for a bride from the creatures ...,"Sauti continued, 'The Muni, having said so unt...","""My ancestors, afflicted with grief, have dire...",10,26
2,10002,10193,Book01_046,Who informed Vasuki about the Muni's desire fo...,The snakes that had been set upon Jaratkaru's ...,"Sauti continued, 'The Muni, having said so unt...","""Then those snakes that had been set upon Jara...",10,20
3,10003,10193,Book01_046,How did Vasuki respond when he heard about the...,"Vasuki took a maiden, who was decked with orna...","Sauti continued, 'The Muni, having said so unt...","""And the king of the snakes, hearing their wor...",14,22


In [11]:
Question_Len = int(max(len(ques) for ques in df.Question)/4)
Answer_Len = int(max(len(ans) for ans in df.Ref_Answer)/4)
Question_Len,Answer_Len # in Tokens

(34, 222)

# Prepare Dataset and Dataloader

In [12]:
Q_LEN =  256   # Question Length
T_LEN =  32  # Target Length
BATCH_SIZE = 2
# DEVICE = "cuda:0"

In [13]:
print(df.shape)
df.head(3)

(1104, 9)


,Ques_Id,Chunk_Id,Section_Id,Question,Ref_Answer,Chunk,Reference,WordsInQues,WordsInAns
0,10000,10193,Book01_046,Why did the Muni wander over the earth and wee...,The Muni wandered over the earth and wept loud...,"Sauti continued, 'The Muni, having said so unt...","""'O ye, being directed by my ancestors, I am r...",14,25
1,10001,10193,Book01_046,Who did the Muni ask for a bride and why?,The Muni asked for a bride from the creatures ...,"Sauti continued, 'The Muni, having said so unt...","""My ancestors, afflicted with grief, have dire...",10,26
2,10002,10193,Book01_046,Who informed Vasuki about the Muni's desire fo...,The snakes that had been set upon Jaratkaru's ...,"Sauti continued, 'The Muni, having said so unt...","""Then those snakes that had been set upon Jara...",10,20


### Load Base Model for Finetuning

In [14]:
class QA_Dataset(Dataset):
  def __init__(self, tokenizer, dataframe, q_len, t_len):
      self.tokenizer = tokenizer
      self.q_len = q_len
      self.t_len = t_len
      self.data = dataframe
      self.questions = self.data["Question"]
      self.context = self.data["Chunk"]
      self.answer = self.data['Ref_Answer']

  def __len__(self):
      return len(self.questions)

  def __getitem__(self, idx):
      question = self.questions[idx]
      context = self.context[idx]
      answer = self.answer[idx]

      question_tokenized = self.tokenizer(question, context, max_length=self.q_len, padding="max_length",
                                                  truncation=True, pad_to_max_length=True, add_special_tokens=True)
      answer_tokenized = self.tokenizer(answer, max_length=self.t_len, padding="max_length",
                                        truncation=True, pad_to_max_length=True, add_special_tokens=True)

      labels = torch.tensor(answer_tokenized["input_ids"], dtype=torch.long)
      labels[labels == 0] = -100

      return {
          "input_ids": torch.tensor(question_tokenized["input_ids"], dtype=torch.long),
          "attention_mask": torch.tensor(question_tokenized["attention_mask"], dtype=torch.long),
          "labels": labels,
          "decoder_attention_mask": torch.tensor(answer_tokenized["attention_mask"], dtype=torch.long)
      }

In [ ]:
TOKENIZER = T5TokenizerFast.from_pretrained("t5-base")
MODEL = T5ForConditionalGeneration.from_pretrained("t5-base", return_dict=True)
MODEL.to(DEVICE)
OPTIMIZER = Adam(MODEL.parameters(), lr=0.00001)

In [17]:
# Dataloader

train_data, val_data = train_test_split(df, test_size=0.2, random_state=42)

train_sampler = RandomSampler(train_data.index)
val_sampler = RandomSampler(val_data.index)

qa_dataset = QA_Dataset(TOKENIZER, df, Q_LEN, T_LEN)

train_loader = DataLoader(qa_dataset, batch_size=BATCH_SIZE, sampler=train_sampler)
val_loader = DataLoader(qa_dataset, batch_size=BATCH_SIZE, sampler=val_sampler)

# Start finetune (Training)

In [ ]:
train_loss = 0
val_loss = 0
train_batch_count = 0
val_batch_count = 0

for epoch in range(10):
    MODEL.train()
    for batch in tqdm(train_loader, desc="Training batches"):
        input_ids = batch["input_ids"].to(DEVICE)
        attention_mask = batch["attention_mask"].to(DEVICE)
        labels = batch["labels"].to(DEVICE)
        decoder_attention_mask = batch["decoder_attention_mask"].to(DEVICE)

        outputs = MODEL(
                          input_ids=input_ids,
                          attention_mask=attention_mask,
                          labels=labels,
                          decoder_attention_mask=decoder_attention_mask
                        )

        OPTIMIZER.zero_grad()
        outputs.loss.backward()
        OPTIMIZER.step()
        train_loss += outputs.loss.item()
        train_batch_count += 1

    #Evaluation
    MODEL.eval()
    for batch in tqdm(val_loader, desc="Validation batches"):
        input_ids = batch["input_ids"].to(DEVICE)
        attention_mask = batch["attention_mask"].to(DEVICE)
        labels = batch["labels"].to(DEVICE)
        decoder_attention_mask = batch["decoder_attention_mask"].to(DEVICE)

        outputs = MODEL(
                          input_ids=input_ids,
                          attention_mask=attention_mask,
                          labels=labels,
                          decoder_attention_mask=decoder_attention_mask
                        )

        OPTIMIZER.zero_grad()
        outputs.loss.backward()
        OPTIMIZER.step()
        val_loss += outputs.loss.item()
        val_batch_count += 1

    print(f"{epoch+1}/{2} -> Train loss: {train_loss / train_batch_count}\tValidation loss: {val_loss/val_batch_count}")

# Save/ Load Finetuned (trained) Model

In [15]:
model_path = "/content/drive/MyDrive/HBQA/t5qa_model"
token_path = "/content/drive/MyDrive/HBQA/t5qa_tokenizer"

## Save Finetuned (trained) Model

In [22]:
MODEL.save_pretrained(model_path)
TOKENIZER.save_pretrained(token_path)

# Saved files
"""('qa_tokenizer/tokenizer_config.json',
 'qa_tokenizer/special_tokens_map.json',
 'qa_tokenizer/spiece.model',
'qa_tokenizer/added_tokens.json',
'qa_tokenizer/tokenizer.json')"""

"('qa_tokenizer/tokenizer_config.json',\n 'qa_tokenizer/special_tokens_map.json',\n 'qa_tokenizer/spiece.model',\n'qa_tokenizer/added_tokens.json',\n'qa_tokenizer/tokenizer.json')"

## Load Model from Memory

In [ ]:
from transformers import T5ForConditionalGeneration, T5Tokenizer

# Load the corresponding tokenizer
TOKENIZER = T5Tokenizer.from_pretrained(token_path)

# Load the pre-trained T5 model
MODEL = T5ForConditionalGeneration.from_pretrained(model_path)
MODEL.to(DEVICE)

In [ ]:
# input_text = "Translate the following English text to French: 'Hello, how are you?'"

# # Tokenize the input text
# input_ids = tokenizer.encode(input_text, return_tensors='pt')
# input_ids = input_ids.to(DEVICE)

# # Generate translated text
# translated_ids = MODEL.generate(input_ids)

# # Decode the generated IDs back to text
# translated_text = TOKENIZER.decode(translated_ids[0], skip_special_tokens=True)

# print("Translated Text:", translated_text)


# Predict Answers from t5 model

In [17]:
def predict_answer(context, question):

    inputs = TOKENIZER(question, context, max_length=Q_LEN, padding="max_length", truncation=True, add_special_tokens=True)

    input_ids = torch.tensor(inputs["input_ids"], dtype=torch.long).to(DEVICE).unsqueeze(0)
    attention_mask = torch.tensor(inputs["attention_mask"], dtype=torch.long).to(DEVICE).unsqueeze(0)

    outputs = MODEL.generate(input_ids=input_ids, attention_mask=attention_mask)

    predicted_answer = TOKENIZER.decode(outputs.flatten(), skip_special_tokens=True)
    return predicted_answer

### Check Answers

In [17]:
import random
qno=[]
N=5

for i in range(N):
    qno.append(random.randint(0,df.shape[0] ) )

pred_answers=[]
ref_answers=[]
question=[]

for i in qno:
    chunk    = df.iloc[i]['Chunk'],
    ques     = df.iloc[i]['Question']
    ref_ans  = df.iloc[i]['Ref_Answer']

    pred_ans = predict_answer(chunk, ques)

    pred_answers.append(pred_ans)
    ref_answers.append(ref_ans)
    question.append(ques)

    print('Question  :', ques)
    print("Ref Answer:", ref_ans)
    print("Pred Ans  :", pred_ans)
    print('--------')

Question  : Whose voice did the Muni hear reciting the Vedas?
Ref Answer: The Muni heard the voice of his daughter-in-law Adrisyanti, who was pregnant with a child from his son Saktri, reciting the Vedas with the six graces of elocution.
Pred Ans  : The Muni heard the voice of the Lord Krishna reciting the Vedas.
--------
Question  : How did Arjuna's presence and actions on the battlefield impact the spectators?
Ref Answer: The spectators observed Arjuna's swift and continuous shooting of arrows without any apparent interval, and their cheers mingled with loud shouts.
Pred Ans  : Arjuna's presence and actions on the battlefield had an impact on the spectators.
--------
Question  : Who did Surya (Vivaswat) decide to bestow Tapati in marriage to?
Ref Answer: Surya decided to bestow Tapati in marriage to King Samvarana, the son of Riksha.
Pred Ans  : Surya (Vivaswat) decided to bestow Tapati in marriage to
--------
Question  : How did Satyavati try to persuade Bhishma to marry and continu

### Predict All Answer

In [18]:
import random
pred_answers=[]
ref_answers=[]
questions=[]
for i in range(df.shape[0]):
    chunk    = df.iloc[i]['Chunk'],
    ques     = df.iloc[i]['Question']
    ref_ans  = df.iloc[i]['Ref_Answer']

    pred_ans = predict_answer(chunk, ques)

    pred_answers.append(pred_ans)
    ref_answers.append(ref_ans)
    questions.append(ques)

    # print('Question  :', ques)
    # print("Ref Answer:", ref_ans)
    # print("Pred Ans  :", pred_ans)
    # print('--------')
    print(f"Predicting ans for question {df.iloc[i]['Ques_Id']}")
    df.loc[i,'Pred_Answer'] = pred_ans

Predicting ans for question 10000
Predicting ans for question 10001
Predicting ans for question 10002
Predicting ans for question 10003
Predicting ans for question 10004
Predicting ans for question 10005
Predicting ans for question 10006
Predicting ans for question 10007
Predicting ans for question 10008
Predicting ans for question 10009
Predicting ans for question 10010
Predicting ans for question 10011
Predicting ans for question 10012
Predicting ans for question 10013
Predicting ans for question 10014
Predicting ans for question 10015
Predicting ans for question 10016
Predicting ans for question 10017
Predicting ans for question 10018
Predicting ans for question 10019
Predicting ans for question 10020
Predicting ans for question 10021
Predicting ans for question 10022
Predicting ans for question 10023
Predicting ans for question 10024
Predicting ans for question 10025
Predicting ans for question 10026
Predicting ans for question 10027
Predicting ans for question 10028
Predicting ans

In [20]:
df.to_csv(datapath + '09.11-t5Predicted_ans.csv')
# df= pd.read_csv(datapath + '11.1-t5Predicted_ans.csv')

In [21]:
def calculate_score(ref_answer, predicted_answer):

  pred_answer_ids = TOKENIZER.encode(predicted_answer, return_tensors='pt')[0][0]
  pred_answer_ids = pred_answer_ids.to(DEVICE)

  ref_answer_ids = TOKENIZER.encode(ref_answer, return_tensors='pt')[0][0]
  ref_answer_ids = pred_answer_ids.to(DEVICE)

  bleu = evaluate.load("google_bleu")
  bleu_score1  = bleu.compute(predictions=[predicted_answer], references=[ref_answer])

  # squad = evaluate.load("squad")
  glue_qqp = evaluate.load('glue', 'qqp')

  glue_qqp_score1 = glue_qqp.compute(predictions=[pred_answer_ids],
                      references=[ref_answer_ids])

  return bleu_score1, glue_qqp_score1 #squad_score1 #bleu_score1#, squad_score1, glue_score1

In [28]:
df1 = pd.DataFrame(columns = ['Ques_Id','BLEU_Score','GLUE_Acc','GLUE_F1'])

In [31]:
bleu_score=[]
glue_qqp_score=[]
from datasets import load_metric

N= df.shape[0]

for i in range(N):
  # bleu_score1 = calculate_score( df1.iloc[i]['ref_answer'],df1.iloc[i]['pred_answer'])
  ques_id = df.loc[i,'Ques_Id']
  ref_ans  = df.loc[i,'Ref_Answer']
  pred_ans = df.loc[i,'Pred_Answer']

  bleu, glue = calculate_score(ref_ans, pred_ans)
  print(bleu,glue,ques_id)

  bleu_score.append(bleu['google_bleu'])
  glue_qqp_score.append(glue['accuracy'])
  glue_qqp_score.append(glue['f1'])

  df1.loc[i] = (ques_id, bleu['google_bleu'], glue['accuracy'], glue['f1'] )

{'google_bleu': 0.336734693877551} {'accuracy': 1.0, 'f1': 0.0} 10000
{'google_bleu': 0.2549019607843137} {'accuracy': 1.0, 'f1': 0.0} 10001
{'google_bleu': 0.0} {'accuracy': 1.0, 'f1': 1.0} 10002
{'google_bleu': 0.06382978723404255} {'accuracy': 1.0, 'f1': 0.0} 10003
{'google_bleu': 0.2358490566037736} {'accuracy': 1.0, 'f1': 0.0} 10004
{'google_bleu': 0.4782608695652174} {'accuracy': 1.0, 'f1': 0.0} 10005
{'google_bleu': 0.058823529411764705} {'accuracy': 1.0, 'f1': 0.0} 10006
{'google_bleu': 0.0} {'accuracy': 1.0, 'f1': 1.0} 10007
{'google_bleu': 0.1320754716981132} {'accuracy': 1.0, 'f1': 0.0} 10008
{'google_bleu': 0.01694915254237288} {'accuracy': 1.0, 'f1': 0.0} 10009
{'google_bleu': 0.10204081632653061} {'accuracy': 1.0, 'f1': 0.0} 10010
{'google_bleu': 0.46938775510204084} {'accuracy': 1.0, 'f1': 0.0} 10011
{'google_bleu': 0.14935064935064934} {'accuracy': 1.0, 'f1': 0.0} 10012
{'google_bleu': 0.3979591836734694} {'accuracy': 1.0, 'f1': 0.0} 10013
{'google_bleu': 0.3} {'accurac

In [39]:
df1.sample(5)

,Ques_Id,BLEU_Score,GLUE_Acc,GLUE_F1
121,10121.0,0.520000,1.0,0.0
576,10576.0,0.347826,1.0,0.0
944,10944.0,0.217949,1.0,0.0
1021,11021.0,0.077778,1.0,0.0
921,10921.0,0.088889,1.0,0.0


In [40]:
df1.to_csv(datapath + '09.12-t5Predicted_Ans_Score.csv')

In [41]:
df1[[ 'BLEU_Score', 'GLUE_Acc', 'GLUE_F1']].mean()

BLEU_Score    0.169266
GLUE_Acc      1.000000
GLUE_F1       0.022645
dtype: float64

In [ ]:
# sst2, mnli, mnli_mismatched, mnli_matched, qnli, rte, wnli, cola,stsb, mrpc, qqp, and hans.

In [ ]:
from evaluate import load
glue_metric = load('glue', 'mrpc')  # 'mrpc' or 'qqp'
references = [0, 1]
predictions = [0, 1]
results = glue_metric.compute(predictions=predictions, references=references)
print(results)

In [ ]:
from evaluate import load
glue_metric = load('glue', 'stsb')
references = [0., 1., 2., 3., 4., 5.]
predictions = [-10., -11., -12., -13., -14., -15.]
results = glue_metric.compute(predictions=predictions, references=references)
print(results)

In [ ]:
from evaluate import load
glue_metric = load('glue', 'cola')
references = [0, 1]
predictions = [1, 1]
results = glue_metric.compute(predictions=predictions, references=references)
results